The following Assignment focuses on building a sentiment analyzer. It should be noted that the huggingface Space hardware could only Accomodate RapidApi's function due to package limitations.

In [8]:
#Installing Necessary Packages and Configuring ngrok with Authentication Token
!pip install fastapi nest-asyncio pyngrok uvicorn
!ngrok config add-authtoken 2fNQ7AEiwJ6tGGOxYyReXF9rFKu_6Pmy7vMkJCuNYQiesnH8S

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
#Importing all of the requirements
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import requests
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-002f20429327>", line 37, in <cell line: 37>
    uvicorn.run(app, port=8000)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/u

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

The following are the prompts you can test to get the said results


Positive: today was a very good day

Neutral: Hello! (only applicable for RapidApi function)

Negative: Mr. Stark, I don't feel so good

In [7]:
app = FastAPI()

#bringing in my API form RapidAPI- My first endpoint
url = "https://text-sentiment-analyzer-api1.p.rapidapi.com/sentiment"

#setting up function that takes the user input - RapidApi
def call_sentiment_api(user_input):
    # Creating a payload using the user input
    payload = f"-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"text\"\r\n\r\n{user_input}\r\n-----011000010111000001101001--\r\n\r\n"
    headers = {
        "x-rapidapi-key": "c22f916d7emsh84b9036bb10a1b9p1f1cecjsne5deda81b82a",
        "x-rapidapi-host": "text-sentiment-analyzer-api1.p.rapidapi.com",
        "Content-Type": "multipart/form-data; boundary=---011000010111000001101001"
    }

    response = requests.post(url, data=payload, headers=headers)
    print(response.json())
    return response.json()


#Setting up function for Hugging Face API
def sentiment_model_hf(user_input):
  inputs = tokenizer(user_input, return_tensors="pt")
  with torch.no_grad():
    logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
  return model.config.id2label[predicted_class_id]



#home page endpoint
@app.get('/')
async def Home():


  return "Hello There, This is my Assingment 2"

#Analyze endpoint that calls my sentiment function
@app.get('/Analyze/{user_input}')
async def Analyze(user_input):


  return call_sentiment_api(user_input)

#Analyze_hf endpoint that calls my hf Sentiment funtion
@app.get('/sentiment_hf/{user_input}')
async def sentiment_hf(user_input):
  return sentiment_model_hf(user_input)

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://04bf-34-80-246-186.ngrok-free.app


INFO:     Started server process [481]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [481]


KeyboardInterrupt: 